# Real Estate Market Selection Model

<b> Goal of Model and Analysis </b>
    
The goal of this analysis is to help inform which city Mehrmah and I decide to buy an investment property in.  The output of the analysis will be a ranked list of cities with forecasted 5YR and 10YR appreciation values.

In order to reduce the risk of us losing money, a strict criteria for our next investment property is that it will be cashflow neutral or better, however we recognize that appreciation will likely be the primary way to get strong returns.  Therefore, our objective function is to maximize appreciation under the constraint that the property cashflows with a 20% downpayment. 

Given a property’s ability to cashflow requires analysis on the individual property, we will first do the analysis to determine which markets to look at individual properties in.  


In [1]:
import pandas as pd

# Data Imports and Reformating

In [2]:
# import zipcode to FIPS mapping
zip_mapping = pd.read_csv('ZIP-COUNTY-FIPS_2018-03.csv')
zip_mapping.rename(columns={"ZIP": "zipcode", "STCOUNTYFP": "fips", "CITY": "city_mapped", "STATE": "state_mapped", "COUNTYNAME":"county_mapped", "CLASSFP":"classfp"}, inplace = True)

In [3]:
# import 2010 population by zipcode data for ZHVI weighting
pop_weights = pd.read_csv('Population/Census+Population+By+Zipcode+(ZCTA).csv')

In [4]:
# import and reformat ZHVI data
zhvi = pd.read_csv('ZHVI/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')
zhvi = pd.melt(zhvi, id_vars=['RegionID','SizeRank','RegionName','RegionType','StateName','State','City','Metro','CountyName'])
zhvi.rename(columns={"RegionName": "zipcode", "StateName": "state", "Metro": "msa", "CountyName": "county", "variable":"month", "value":"zhvi"}, inplace = True)
zhvi['month'] = pd.to_datetime(zhvi.month, infer_datetime_format=True)
zhvi['year'] = zhvi.month.dt.year
zhvi = zhvi[["zipcode","state","msa","county","year","zhvi"]].loc[zhvi.month.dt.month == 1]
zhvi = zhvi.merge(zip_mapping, how = 'left',left_on = 'zipcode',right_on = 'zipcode')


In [5]:
# import and reformat GDP data
gdp_county = pd.read_csv('GDP/CAGDP2__ALL_AREAS_2001_2019.csv', skipfooter = 4)
gdp_county.GeoFIPS = gdp_county.GeoFIPS.str.replace('"', '')
gdp_county = pd.melt(gdp_county, id_vars=['GeoFIPS','GeoName','Region','TableName','LineCode','IndustryClassification','Description','Unit'])
gdp_county.rename(columns={"GeoFIPS": "fips", "GeoName": "county", "Description": "metric", "variable": "year", "value":"gdp"}, inplace = True)
gdp_county = gdp_county[["fips","county","metric","year","gdp"]]
gdp_county = gdp_county[gdp_county.gdp.str.isnumeric() == True]  #dropping empty values
gdp_county.gdp = pd.to_numeric(gdp_county.gdp)
gdp_county = gdp_county.pivot_table(index=["fips","county","year"], columns="metric")['gdp'].reset_index()
gdp_county.columns.name = None
gdp_county.fips = gdp_county.fips.str.replace(' ','')
gdp_county.fips = pd.to_numeric(gdp_county.fips)
gdp_county.year = gdp_county.year.astype(int)


/code/venvs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  


In [6]:
# import and reformat 2000 Population data
pop_2000 = pd.read_csv('Population/DECENNIALSF12000.P001_data_with_overlays_2021-07-23T112021.csv')

#all FIPS start with 0500000US in this dataset.  cutting that part out
pop_2000.GEO_ID = pop_2000.GEO_ID.str.replace('0500000US','')
pop_2000 = pop_2000[~(pop_2000.GEO_ID == 'id')] #filtering out header row
pop_2000.GEO_ID = pd.to_numeric(pop_2000.GEO_ID)
pop_2000["year"] = 2000
pop_2000.rename(columns={"GEO_ID": "fips", "NAME": "county", "P001001": "population"}, inplace = True)


In [7]:
# import and reformat 2010 to 2020 Population data
pop_2010_2020 = pd.read_csv('Population/PEPPOP2019.PEPANNRES_data_with_overlays_2021-08-08T193054.csv')
pop_2010_2020.head()
pop_2010_2020 = pop_2010_2020[~pop_2010_2020.DATE_CODE.str.contains('Census')]
pop_2010_2020 = pop_2010_2020[~pop_2010_2020.DATE_CODE.str.contains('base')]
pop_2010_2020.DATE_CODE = pop_2010_2020.DATE_CODE.str.slice(stop = 8)
pop_2010_2020.DATE_CODE = pop_2010_2020.DATE_CODE.str.slice(start = 4)
pop_2010_2020 = pop_2010_2020[~(pop_2010_2020.GEO_ID == 'id')] #filtering out header row
pop_2010_2020.GEO_ID = pop_2010_2020.GEO_ID.str.replace('0500000US','')
pop_2010_2020.rename(columns={"GEO_ID": "fips", "NAME": "county","DATE_CODE": "year", "POP": "population"}, inplace = True)
pop_2010_2020.fips = pd.to_numeric(pop_2010_2020.fips)
pop_2010_2020.year = pd.to_numeric(pop_2010_2020.year)
pop_2010_2020.population = pd.to_numeric(pop_2010_2020.population)


In [8]:
#merge the two population datasets together
population = pd.concat([pop_2000, pop_2010_2020])

#linearly interpolate missing years from 2000 to 2010
population_start = population[population.year ==2000][['fips','population']]
population_start.rename(columns={"population":"2000_pop"}, inplace = True)

population_end = population[population.year ==2010][['fips','population']]
population_end.rename(columns={"population":"2010_pop"}, inplace = True)

population_slope = population_start.merge(population_end, how = 'left', on = 'fips')
population_slope["slope"] = (pd.to_numeric(population_slope["2010_pop"]) - pd.to_numeric(population_slope["2000_pop"]))/10
population_slope

fips = pd.DataFrame(population.fips.unique(), columns = ['fips'])
years = pd.DataFrame({'year': [x for x in range(2001, 2010)]})

fips['key'] = 1
years['key'] = 1

interpolated_pop = pd.merge(fips, years, on ='key').drop("key", 1)
interpolated_pop = interpolated_pop.merge(population_slope, how = 'left', on = 'fips')
interpolated_pop["baseline_year"] = 2000
interpolated_pop["2000_pop"] = pd.to_numeric(interpolated_pop["2000_pop"])
interpolated_pop["2010_pop"]= pd.to_numeric(interpolated_pop["2010_pop"])
interpolated_pop["population"] = interpolated_pop["2000_pop"] + (interpolated_pop["year"]-interpolated_pop["baseline_year"])*interpolated_pop["slope"]

#15 missing fips due to 2000 population data not existing
print(interpolated_pop[interpolated_pop.population.isnull()].fips.nunique())
interpolated_pop = interpolated_pop[["fips","year","population"]]

population_w_estimates = pd.concat([population[["fips","year","population"]], interpolated_pop]) 

#df['key'] = 1
#df1['key'] = 1
#result = pd.merge(df, df1, on ='key').drop("key", 1)

15


In [39]:
# import rent data
rent = pd.read_csv("Zip_ZORI_AllHomesPlusMultifamily_SSA.csv")

# Preparing ZHVI Data and Aggregating to the FIPS Level

In [10]:
zhvi = zhvi.merge(pop_weights, how = 'left',left_on = 'zipcode',right_on = 'Zip Code ZCTA')
zhvi['zhvi_pop_prod'] = zhvi['Census Population 2010']*zhvi['zhvi']

In [11]:
#Filter to dates with well populated ZHVI data and GDP data
zhvi = zhvi[zhvi.year > 2000]
zhvi = zhvi[zhvi.year < 2020]
zhvi['missing'] = zhvi.zhvi.isnull()
print(zhvi[zhvi.missing == True]['Census Population 2010'].sum()/zhvi['Census Population 2010'].sum())

#make a list of zipcodes with partial data to exclude from the analysis
zipmissing = zhvi.groupby('zipcode', as_index = False).missing.mean()
print(zipmissing[zipmissing.missing > 0].shape[0] / zipmissing.shape[0])
zipmissing = zipmissing[zipmissing.missing>0].zipcode
zhvi = zhvi[~zhvi.zipcode.isin(zipmissing)]

0.05319918337346067
0.39723717491406707


In [12]:
#Aggregate ZHVI from the zipcode level up to the FIPS level, weighting by 2010 population
zhvi_fips_agg = zhvi.groupby(['fips','year'], as_index = False).agg({'zhvi': 'mean','zhvi_pop_prod': 'sum', 'Census Population 2010': 'sum'})
zhvi_fips_agg['zhvi_weighted'] = zhvi_fips_agg['zhvi_pop_prod'] / zhvi_fips_agg['Census Population 2010']
zhvi_fips_agg.fips = zhvi_fips_agg.fips.astype(int)


In [13]:
# population and therefore weighted ZHVI is missing for 1 FIPS -filter to only data that mapped to 2010 population data
print(zhvi_fips_agg[zhvi_fips_agg.zhvi_weighted.isnull()].fips.nunique())
zhvi_fips_agg.zhvi_weighted.isnull().sum()/zhvi_fips_agg.shape[0]
zhvi_fips_agg = zhvi_fips_agg[zhvi_fips_agg.zhvi_weighted.isnull()==False]

1


# Joining ZHVI Data to Other Data Sources

In [14]:
# Merge ZHVI data with GDP data.  2% of FIPS don't have matching GDP data, filter those out
df_agg = zhvi_fips_agg.merge(gdp_county, how = 'left', on = ['fips','year'])
print(df_agg["All industry total"].isnull().sum()/df_agg.shape[0])
df_agg = df_agg[df_agg["All industry total"].isnull()==False]

0.02125124131082423


In [15]:
# Merge ZHVI data with population data
df_agg = df_agg.merge(population_w_estimates, how = 'left', on = ['fips','year'])
print(df_agg["population"].isnull().sum()/df_agg.shape[0])
df_agg = df_agg[~df_agg["population"].isnull()]

0.00020292207792207794


# Preparing data for export

In [17]:
# make a function to prepare the target feature desired (ie appreciation)
def target_prep(timeseries_df, start_year, lookforward, target_variable):
    baseline_df = timeseries_df[timeseries_df.year == start_year]
    lookforward_df = timeseries_df[timeseries_df.year == start_year + lookforward]
    df_out = baseline_df[["fips", target_variable]]
    df_out = df_out.merge(lookforward_df[['fips',target_variable]], how = 'left', on = 'fips')
    df_out[target_variable] = df_out.iloc[:, [2]].values/df_out.iloc[:, [1]].values
    df_out = df_out[["fips", target_variable]]
    df_out.columns = ['fips', target_variable + '_growth']
    return df_out

In [18]:
#function to calculate growth rate features over X year lookback

def growth_feature_prep(timeseries_df, start_year, look_back, cols_to_adj):
    baseline_df = timeseries_df[timeseries_df.year == start_year]
    lookback_df = timeseries_df[timeseries_df.year == start_year - look_back]

    df_out = baseline_df[["fips"]]
    
    for col in cols_to_adj:
        temp = baseline_df[['fips',col]]
        temp = temp.merge(lookback_df[['fips',col]], how = 'left', on = 'fips')
        temp[col] = temp.iloc[:, [1]].values/temp.iloc[:, [2]].values
        temp = temp[['fips',col]]
        temp.columns = ['fips', col + '_growth']
        df_out = df_out.merge(temp, how = 'left', on= 'fips')        
    return df_out


In [19]:
cols_to_adj = [#'   Accommodation and food services',
               #'   Administrative and support and waste management and remediation services',
               #'   Arts, entertainment, and recreation',
               #'   Durable goods manufacturing', 
               #'   Educational services',
               #'   Finance and insurance', 
               #'   Health care and social assistance',
               #'   Management of companies and enterprises',
               #'   Nondurable goods manufacturing',
               #'   Professional, scientific, and technical services',
               #'   Real estate and rental and leasing',
               #'  Agriculture, forestry, fishing and hunting',
               #'  Arts, entertainment, recreation, accommodation, and food services',
               '  Construction',
               #'  Educational services, health care, and social assistance',
               '  Finance, insurance, real estate, rental, and leasing',
               #'  Information', 
               '  Manufacturing',
               '  Mining, quarrying, and oil and gas extraction',
               #'  Other services (except government and government enterprises)',
               #'  Professional and business services', 
               '  Retail trade',
               #'  Transportation and warehousing', 
               #'  Utilities', 
               #'  Wholesale trade',
               ' Private industries', 
               'All industry total',
               'Government and government enterprises',
               #'Manufacturing and information', 
               #'Natural resources and mining',
               #'Private goods-producing industries 2/',
               #'Private services-providing industries 3/', 
               #'Trade',
               #'Transportation and utilities',
               'population'
]

In [20]:
fips_map = gdp_county.groupby(['fips','county'], as_index = False).year.count()
fips_map = fips_map[['fips','county']]

In [21]:
df_agg.columns

Index(['fips', 'year', 'zhvi', 'zhvi_pop_prod', 'Census Population 2010',
       'zhvi_weighted', 'county', '   Accommodation and food services',
       '   Administrative and support and waste management and remediation services',
       '   Arts, entertainment, and recreation',
       '   Durable goods manufacturing', '   Educational services',
       '   Finance and insurance', '   Health care and social assistance',
       '   Management of companies and enterprises',
       '   Nondurable goods manufacturing',
       '   Professional, scientific, and technical services',
       '   Real estate and rental and leasing',
       '  Agriculture, forestry, fishing and hunting',
       '  Arts, entertainment, recreation, accommodation, and food services',
       '  Construction',
       '  Educational services, health care, and social assistance',
       '  Finance, insurance, real estate, rental, and leasing',
       '  Information', '  Manufacturing',
       '  Mining, quarrying, and o

In [22]:
baseline_features = ['fips'
                    ,'Census Population 2010',
                     'zhvi_weighted',
                    #'   Accommodation and food services',
                    #'   Administrative and support and waste management and remediation services',
                    #'   Arts, entertainment, and recreation',
                    #'   Durable goods manufacturing', 
                    #'   Educational services',
                    #'   Finance and insurance', 
                    #'   Health care and social assistance',
                    #'   Management of companies and enterprises',
                    #'   Nondurable goods manufacturing',
                    #'   Professional, scientific, and technical services',
                    #'   Real estate and rental and leasing',
                    #'  Agriculture, forestry, fishing and hunting',
                    #'  Arts, entertainment, recreation, accommodation, and food services',
                    #'  Construction',
                    #'  Educational services, health care, and social assistance',
                    #'  Finance, insurance, real estate, rental, and leasing',
                    #'  Information', 
                    #'  Manufacturing',
                    #'  Mining, quarrying, and oil and gas extraction',
                    #'  Other services (except government and government enterprises)',
                    #'  Professional and business services', 
                    #'  Retail trade',
                    #'  Transportation and warehousing', 
                    #'  Utilities', 
                    #'  Wholesale trade',
                    ' Private industries', 
                    'All industry total',
                    'Government and government enterprises',
                    #'Manufacturing and information', 
                    #'Natural resources and mining',
                    #'Private goods-producing industries 2/',
                    #'Private services-providing industries 3/', 
                    #'Trade',
                     #'Transportation and utilities',
                     'population']

In [93]:
#Preparing Training & Testing Dataset
start_year = 2004
lookforward = 3
look_back = 3


df_target = target_prep(df_agg, start_year = start_year, lookforward = lookforward, target_variable = 'zhvi_weighted')
df_growth_features = growth_feature_prep(df_agg, start_year = start_year, look_back = look_back, cols_to_adj = cols_to_adj)
baseline_df = df_agg[df_agg.year == start_year]
df = df_target.merge(df_growth_features, how = 'left', on = 'fips')
df = df.merge(fips_map, how = 'left',on = 'fips')
df = df.merge(baseline_df[baseline_features], how = 'left',on = 'fips')

df_train_test = df

df_train_test.to_csv('df_train_test.csv', index = False)

/code/venvs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in true_divide
  if sys.path[0] == '':
/code/venvs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


In [92]:
# Preparing Validation Dataset
start_year = 2016
lookforward = 3
look_back = 3

df_target = target_prep(df_agg, start_year = start_year, lookforward = lookforward, target_variable = 'zhvi_weighted')
df_growth_features = growth_feature_prep(df_agg, start_year = start_year, look_back = look_back, cols_to_adj = cols_to_adj)
baseline_df = df_agg[df_agg.year == start_year]
df = df_target.merge(df_growth_features, how = 'left', on = 'fips')
df = df.merge(fips_map, how = 'left',on = 'fips')
df = df.merge(baseline_df[baseline_features], how = 'left',on = 'fips')

df_val = df

df_val.to_csv('df_validation.csv', index = False)

/code/venvs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in true_divide
  if sys.path[0] == '':
/code/venvs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


In [94]:
# Preparing Scoring Dataset
start_year = 2019
#lookforward = 3
look_back = 3

#df_target = target_prep(df_agg, start_year = start_year, lookforward = lookforward, target_variable = 'zhvi_weighted')
df_growth_features = growth_feature_prep(df_agg, start_year = start_year, look_back = look_back, cols_to_adj = cols_to_adj)
baseline_df = df_agg[df_agg.year == start_year]
#df = df_target.merge(df_growth_features, how = 'left', on = 'fips')
df = df_growth_features.merge(fips_map, how = 'left',on = 'fips')
df = df.merge(baseline_df[baseline_features], how = 'left',on = 'fips')

df_score = df

df_score.to_csv('df_score.csv', index = False)


/code/venvs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in true_divide
  if sys.path[0] == '':
/code/venvs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


In [25]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True)
missing_value_df.head(50)

,column_name,percent_missing
fips,fips,0.000000
All industry total,All industry total,0.000000
Private industries,Private industries,0.000000
zhvi_weighted,zhvi_weighted,0.000000
Census Population 2010,Census Population 2010,0.000000
county,county,0.000000
population_growth,population_growth,0.000000
Government and government enterprises_growth,Government and government enterprises_growth,0.000000
All industry total_growth,All industry total_growth,0.000000
Private industries_growth,Private industries_growth,0.000000


In [38]:
df_agg[df_agg.county.str.contains('San Fran')].groupby(['year']).agg({'zhvi_weighted':'sum'}).to_csv('sf_growth.csv')

# QA

In [42]:
# Check that values in the validation dataset match expectations vs the raw data
#Looks good for GDP data comparing 2016 to 2013
df_val.head()

,fips,zhvi_weighted_growth,Construction_growth,"Finance, insurance, real estate, rental, and leasing_growth",Manufacturing_growth,"Mining, quarrying, and oil and gas extraction_growth",Retail trade_growth,Private industries_growth,All industry total_growth,Government and government enterprises_growth,population_growth,county,Census Population 2010,zhvi_weighted,Private industries,All industry total,Government and government enterprises,population
0,1001,1.082575,1.051956,1.080042,1.461403,1.163204,1.029792,1.183760,1.177357,1.140960,1.00943,"Autauga, AL",108787.0,133298.048379,1467403.0,1716235.0,248832.0,55243
1,1003,1.196052,1.549400,1.082270,1.132365,1.411830,1.181364,1.172457,1.165286,1.114800,1.06525,"Baldwin, AL",200224.0,166219.255019,5447808.0,6183519.0,735711.0,207601
2,1005,1.263707,1.328328,1.057611,0.810301,1.228594,1.016105,0.905310,0.918274,1.000026,0.958013,"Barbour, AL",20591.0,83345.580254,650008.0,763866.0,113858.0,25806
3,1007,1.114827,1.019554,1.066709,0.887599,0.344712,1.059316,1.012150,1.022325,1.058336,1.00289,"Bibb, AL",58454.0,157687.981592,304988.0,395153.0,90166.0,22586
4,1009,1.136720,1.116591,1.064714,0.884030,0.467579,1.055315,0.988232,0.995736,1.034130,0.997831,"Blount, AL",140642.0,146741.044532,727561.0,876333.0,148771.0,57494


In [88]:
df_train_test.head()

,fips,zhvi_weighted_growth,Construction_growth,"Finance, insurance, real estate, rental, and leasing_growth",Manufacturing_growth,"Mining, quarrying, and oil and gas extraction_growth",Retail trade_growth,Private industries_growth,All industry total_growth,Government and government enterprises_growth,population_growth,county,Census Population 2010,zhvi_weighted,Private industries,All industry total,Government and government enterprises,population
0,1001,1.209146,0.694951,1.236581,1.016480,1.691146,1.244984,1.238746,1.264443,1.438165,1.07437,"Autauga, AL",108787.0,112952.424159,808170.0,946964.0,138793.0,48111.8
1,1003,1.457189,1.226898,1.285057,1.199572,1.305015,1.338502,1.292162,1.290783,1.281135,1.08853,"Baldwin, AL",200224.0,138199.736121,3440210.0,3927738.0,487527.0,157494
2,1005,1.195766,1.099945,1.144663,1.291125,0.992997,1.242456,1.206069,1.192440,1.107652,0.982218,"Barbour, AL",20591.0,68310.832063,663473.0,761416.0,97943.0,28353.6
3,1007,1.169099,1.897191,1.109655,1.194022,0.859638,1.064291,1.242564,1.224252,1.169466,1.02916,"Bibb, AL",58454.0,146716.181356,208214.0,273739.0,65524.0,21643.6
4,1009,1.127905,1.086735,1.146160,NaN,1.982682,1.023986,1.113936,1.126677,1.205297,1.03689,"Blount, AL",140642.0,136578.880683,594727.0,698997.0,104269.0,53564.8


In [91]:
gdp_qa = pd.read_csv('GDP/CAGDP2__ALL_AREAS_2001_2019.csv', skipfooter = 4)
gdp_qa[(gdp_qa.GeoFIPS.str.contains("1001"))][["Description","2001","2004","2013","2016"]]

/code/venvs/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,Description,2001,2004,2013,2016
68,All industry total,748918,946964,1457702,1716235
69,Private industries,652410,808170,1239612,1467403
70,"Agriculture, forestry, fishing and hunting",14756,39394,28355,22791
71,"Mining, quarrying, and oil and gas extraction",1457,2464,14681,17077
72,Utilities,19515,81339,376088,456310
...,...,...,...,...,...
97609,Trade,62825,69162,92745,97884
97610,Transportation and utilities,17624,22520,(D),(D)
97611,Manufacturing and information,921317,654996,1095804,490533
97612,Private goods-producing industries 2/,1019309,817522,1261385,617639


In [80]:
zhvi_zip= pd.read_csv('ZHVI/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')

In [81]:
zipcodes_in_01001 = zip_mapping[zip_mapping.fips == 1001].zipcode.values

In [82]:
zhvi_zip = zhvi_zip[zhvi_zip.RegionName.isin(zipcodes_in_01001)]

In [83]:
zhvi_zip = zhvi_zip.merge(pop_weights, how = 'left', left_on = 'RegionName', right_on = 'Zip Code ZCTA')

In [89]:
zhvi_zip['weighed_zhvi_2019'] = zhvi_zip['2019-01-31']*zhvi_zip['Census Population 2010']
zhvi_zip['weighed_zhvi_2016'] = zhvi_zip['2016-01-31']*zhvi_zip['Census Population 2010']
zhvi_zip['weighed_zhvi_2019'].sum() / zhvi_zip['weighed_zhvi_2016'].sum()

#1.082575

1.0825746902163775

In [90]:
zhvi_zip['weighed_zhvi_2007'] = zhvi_zip['2007-01-31']*zhvi_zip['Census Population 2010']
zhvi_zip['weighed_zhvi_2004'] = zhvi_zip['2004-01-31']*zhvi_zip['Census Population 2010']
zhvi_zip['weighed_zhvi_2007'].sum() / zhvi_zip['weighed_zhvi_2004'].sum()

1.209146

1.209146